<a href="https://colab.research.google.com/github/datrends/dash-plotly/blob/main/Dashboard_Dash_Ingresos_vs_Costos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade plotly
!pip install jupyter-dash

In [2]:
# Importar las bibliotecas necesarias
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

In [3]:
# Cargar los datos desde GitHub
costs_url = 'https://raw.githubusercontent.com/datrends/dash-plotly/main/costs_2022.csv'
revenue_url = 'https://raw.githubusercontent.com/datrends/dash-plotly/main/revenue_2022.csv'
costs_df = pd.read_csv(costs_url)
revenue_df = pd.read_csv(revenue_url)

# Referencia para ordenamiento de meses
month_order_mapping = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

# Limpiar y transformar los datos
def clean_and_transform_data(df, clean_cols, melt_tag):
    # Eliminar las columnas
    df = df.drop(columns=clean_cols, errors='ignore')

    # Limpiar caracteres especiales y convertir a número decimal
    df.replace('[\$,]', '', regex=True, inplace=True)
    df = df.apply(pd.to_numeric, errors='ignore')

    # Anular la dinamización de las columnas de meses, al formato: llave-valor
    df = df.melt(id_vars=['Line Of Business'], var_name='Month', value_name=melt_tag)
    #revenue_df_transposed = revenue_df.melt(id_vars=['Line Of Business'], var_name='Month', value_name='Revenue')

    return df

clean_cols = ['N', 'Expense Item', 'Total']
costs_df = clean_and_transform_data(costs_df, clean_cols, 'Cost')

clean_cols = ['N', 'Client Name', 'Total']
revenue_df = clean_and_transform_data(revenue_df, clean_cols, 'Revenue')

# Obtener valores únicos de "Line Of Business" de ambas fuentes
line_of_business_options = list(set(costs_df['Line Of Business'].unique()) | set(revenue_df['Line Of Business'].unique()))

# Crear el dashboard
app = JupyterDash(__name__)

# Maquetación de la página web
app.layout = html.Div([
    # Título de la web
    html.H1("Comparación de Ingresos y Costos"),

    # Filtro de selección múltiple
    dcc.Dropdown(
        id='line-of-business-selector',
        options=[{'label': i, 'value': i} for i in line_of_business_options],
        value=line_of_business_options,
        multi=True,
        style={'width': '50%'}
    ),

    # Gráfica de comparación ingresos vs. costos
    dcc.Graph(id='revenue-costs-plot')
])

# Callback para actualizar la gráfica
@app.callback(
    Output('revenue-costs-plot', 'figure'),
    [Input('line-of-business-selector', 'value')]
)
def update_plot(selected_line_of_business):
    # Filtrar los datos según la selección de líneas de negocio
    costs_filtered = costs_df[costs_df['Line Of Business'].isin(selected_line_of_business)]
    revenue_filtered = revenue_df[revenue_df['Line Of Business'].isin(selected_line_of_business)]

    # Agrupar por mes, realizar la suma agregada de los valores y el ordenamiento por mes
    costs_grouped = costs_filtered.groupby(['Month']).sum().reset_index()
    costs_grouped["order"] = costs_grouped["Month"].apply(lambda month: month_order_mapping[month])
    costs_grouped = costs_grouped.sort_values(by="order")
    revenue_grouped = revenue_filtered.groupby(['Month']).sum().reset_index()
    revenue_grouped["order"] = revenue_grouped["Month"].apply(lambda month: month_order_mapping[month])
    revenue_grouped = revenue_grouped.sort_values(by="order")

    # Obtener los valores de costos, ingresos y meses
    months_of_year = costs_grouped["Month"].values
    total_cost = costs_grouped["Cost"].values
    total_revenue = revenue_grouped["Revenue"].values

    # Gráfico de barras: ingresos
    fig = go.Figure(
        data=go.Bar(
            x=months_of_year,
            y=total_revenue,
            name="Total revenue",
            marker=dict(color="paleturquoise"),
        )
    )

    # Gráfico de línea: costos
    fig.add_trace(
        go.Scatter(
            x=months_of_year,
            y=total_cost,
            yaxis="y2",
            name="Total cost",
            marker=dict(color="crimson"),
        )
    )

    # Leyenda
    fig.update_layout(
        legend=dict(orientation="h"),
        yaxis=dict(
            title=dict(text="Total revenue"),
            side="left",
            #range=[0, 100000],
        ),
        yaxis2=dict(
            title=dict(text="Total cost"),
            side="right",
            #range=[0, 10],
            overlaying="y",
            tickmode="sync",
        ),
    )

    return fig

/usr/local/lib/python3.10/dist-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [4]:
# Ejecutar la aplicación
if __name__ == '__main__':
  app.run_server(mode='inline')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>